# Carga dados petmar Big Query (BQ)

## Carregar módulos

In [ ]:
from google.colab import auth
from google.cloud import bigquery

In [ ]:
import numpy as np
import pandas as pd
import os
import requests
import glob

## Autenticar projeto

In [ ]:
auth.authenticate_user()

In [ ]:
project_id = 'projetointeligencianegocios'

In [ ]:
client = bigquery.Client(project=project_id)

## Carregar dados

- Dados vendas petmar - automatizado

In [ ]:
url = 'https://api.github.com/repos/RatonesR/pos_graduacao_maua/contents/inteligencia_de_negocios/projeto_final'
lista_de_arquivos = requests.get(url).json()

for arquivo in lista_de_arquivos:
    nome_do_arquivo = arquivo['name']
    url_para_baixar = arquivo['download_url']
    comando = f"wget --no-check-certificate --content-disposition '{url_para_baixar}'"
    os.system(comando)

In [ ]:
padrao_do_arquivo = 'petmar_*.csv'
lista_de_arquivos_encontrados = glob.glob(padrao_do_arquivo)
lista_de_dataframes = []

for nome_arquivo in lista_de_arquivos_encontrados:
    df_temporario = pd.read_csv(nome_arquivo, sep=';')
    lista_de_dataframes.append(df_temporario)

print(lista_de_arquivos_encontrados)

dados_petmar_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)

['petmar_22.csv', 'petmar_24.csv', 'petmar_23.csv']


- Dados vendas petmar - não automatizado

In [ ]:
# ! wget --no-check-certificate --content-disposition 'https://raw.githubusercontent.com/RatonesR/pos_graduacao_maua/refs/heads/main/inteligencia_de_negocios/projeto_final/petmar_22.csv'
# ! wget --no-check-certificate --content-disposition 'https://raw.githubusercontent.com/RatonesR/pos_graduacao_maua/refs/heads/main/inteligencia_de_negocios/projeto_final/petmar_23.csv'
# ! wget --no-check-certificate --content-disposition 'https://raw.githubusercontent.com/RatonesR/pos_graduacao_maua/refs/heads/main/inteligencia_de_negocios/projeto_final/petmar_24.csv'

In [ ]:
# dados_petmar22 = pd.read_csv('petmar_22.csv', sep=';')
# dados_petmar23 = pd.read_csv('petmar_23.csv', sep=';')
# dados_petmar24 = pd.read_csv('petmar_24.csv', sep=';')

In [ ]:
# dados_petmar_consolidado = pd.concat([dados_petmar22, dados_petmar23, dados_petmar24])

## Ajustar dados

In [ ]:
cols = dados_petmar_consolidado.columns.tolist()

In [ ]:
dados_petmar_consolidado = dados_petmar_consolidado[cols].reset_index(drop=True)

In [ ]:
dados_petmar_consolidado = dados_petmar_consolidado.dropna(how='all')

In [ ]:
for coluna in dados_petmar_consolidado.columns:
    if dados_petmar_consolidado[coluna].dtype == 'object':
        dados_petmar_consolidado[coluna] = dados_petmar_consolidado[coluna].astype(str)

## Verificando conformidade

In [ ]:
print(dados_petmar_consolidado.isnull().sum())

cod_empresa          0
cnpj_distribuidor    0
cod_produto          0
nom_produto          0
familia              0
ean                  0
volume               0
valor                0
nota_fiscal          0
dt_venda             0
ano                  0
mes                  0
tipo_venda           0
cnpj_cliente         0
cep_cliente          0
cidade_cliente       0
nom_uf               0
nom_razao            0
nom_bairro           0
nom_vendedor         0
dtype: int64


In [ ]:
dados_petmar_consolidado.dtypes

,0
cod_empresa,object
cnpj_distribuidor,object
cod_produto,int64
nom_produto,object
familia,object
ean,object
volume,int64
valor,int64
nota_fiscal,object
dt_venda,object


In [ ]:
dados_petmar_consolidado.head()

,cod_empresa,cnpj_distribuidor,cod_produto,nom_produto,familia,ean,volume,valor,nota_fiscal,dt_venda,ano,mes,tipo_venda,cnpj_cliente,cep_cliente,cidade_cliente,nom_uf,nom_razao,nom_bairro,nom_vendedor
0,PETMAR,"1,54094E+13",5063,Alizin 10 mL,CA,"7,89752E+12",4,1256,03/01/1970,01/06/2022,2022,6,V,59558881600,11013163,SANTOS,SP,SANDRO HENRIQUE DE CAMARGOS,VILA NOVA,CLAUDIO DA SILVA
1,PETMAR,"1,54094E+13",5063,Alizin 10 mL,CA,"7,89752E+12",5,1258,03/01/1970,01/06/2022,2022,6,V,"1,03637E+13",12327692,JACAREI,SP,VENEZA PET COMERCIO LTDA,JARDIM MESQUITA,CLAUDIO DA SILVA
2,PETMAR,"1,54094E+13",5063,Alizin 10 mL,CA,"7,89752E+12",1,464,03/01/1970,07/06/2022,2022,6,V,"4,19201E+13",11740000,ITANHAEM,SP,MEGA PET SHOP ITANHAEM LTDA,ESTANCIA BEIRA MAR,RODINEI GONÇALVES DA COSTA
3,PETMAR,"1,54094E+13",5750,Allerderm Spot On S.I.S - 2 mL - 6 Pipetas,CA,"7,89752E+12",1,161,02/01/1970,10/01/2022,2022,1,V,"3,30608E+13",11035160,SANTOS,SP,PETLAND PONTA DA PRAIA PET SHOP LTDA,PONTA DA PRAIA,AMANDA GODOY DOS SANTOS
4,PETMAR,"1,54094E+13",5750,Allerderm Spot On S.I.S - 2 mL - 6 Pipetas,CA,"7,89752E+12",1,170,02/01/1970,10/01/2022,2022,1,V,"1,13747E+13",11035170,SANTOS,SP,PET DA PONTA LTDA - ME,APARECIDA,BRUNNO RUTA DA SILVA


## Adicionar dados no BQ

In [ ]:
list(client.list_datasets(project = project_id))

In [ ]:
# cria conjunto de dados (não rodar pela segunda vez)
dataset_ref = bigquery.Dataset(project_id+'.petmar')
dataset_ref = client.create_dataset(dataset_ref)

In [ ]:
table_ref = dataset_ref.table("dados_petmar_consolidado")

In [ ]:
job = client.load_table_from_dataframe(dados_petmar_consolidado, table_ref,
                                       job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job.result()

LoadJob<project=projetointeligencianegocios, location=US, id=a0d034ed-953e-422a-81f0-f1eb71c6f396>